In [91]:
import pandas as pd
import numpy as np

df = pd.read_csv('scaled.csv')

def categorize_revenue(revenue):
    if 'Less than $1 million' in revenue:
        return 1
    elif 'million' in revenue and 'billion' not in revenue and 'Less than $1 million' not in revenue and '$10+ billion' not in revenue:
        return 2
    elif 'million' in revenue and 'billion' in revenue and 'Less than $1 million' not in revenue and '$10+ billion' not in revenue:
        return 3
    elif 'billion' in revenue and 'million' not in revenue and 'Less than $1 million' not in revenue and '$10+ billion' not in revenue:
        return 4
    elif '$10+ billion' in revenue:
        return 5
    else:
        return -9999

# Apply the categorization function to create a new column 'Revenue_Category'
df['Revenue_Category'] = df['Revenue'].apply(categorize_revenue)



In [92]:
#Drop unnamed column that contains row indices
df = df.drop(df.columns[0], axis=1)

In [93]:
from sklearn.impute import KNNImputer

# Identify numerical columns with 'int', 'float', or containing pd.NA values
numerical_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()

# Initialize KNNImputer
imputer = KNNImputer(missing_values=-9999,n_neighbors=10)

# Identify rows where Revenue_Category is 0 or pd.NA
rows_for_imputation = df['Revenue_Category'].isna() | (df['Revenue_Category'] == -9999)

# Perform KNN imputation on NaN values for numerical columns where Revenue_Category is 0 or pd.NA
imputed_values = imputer.fit_transform(df[numerical_columns])

# Copy DataFrame and replace imputed values
df_imputed = df.copy()
df_imputed[numerical_columns] = imputed_values

In [94]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

distinct_values_counts = {}
for column in categorical_columns:
    distinct_values_counts[column] = df[column].nunique()

print("Number of distinct values in categorical columns:")
for column, count in distinct_values_counts.items():
    print(f"'{column}': {count}")


Number of distinct values in categorical columns:
'Job Title': 264
'Salary Estimate': 416
'Job Description': 463
'Company Name': 343
'Location': 200
'Headquarters': 198
'Size': 9
'Type of ownership': 11
'Industry': 60
'Sector': 25
'Revenue': 14
'Competitors': 128
'company_txt': 343
'job_state': 37
'job_simp': 6
'seniority': 3


In [95]:
import re

df = df_imputed
# Function to extract the maximum number from the text
def extract_max_number(text):
    numbers = re.findall(r'\d+', text)  # Find all numbers in the text
    if numbers:
        return max(map(int, numbers))  # Convert to integers and return the maximum
    return None  # Return None if no numbers found

df['Size'] = df['Size'].apply(lambda x: extract_max_number(x))
df['Size'] = np.log(df['Size'])

In [96]:
df['Size'] = df['Size'].fillna(-9999)
numerical_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()
# Identify rows where Size is 0 or pd.NA
rows_for_imputation = df['Size'].isna() | (df['Size'] == -9999)
imputer = KNNImputer(missing_values=-9999,n_neighbors=10)
# Perform KNN imputation on NaN values for numerical columns where Revenue_Category is 0 or pd.NA
imputed_values = imputer.fit_transform(df[numerical_columns])

# Copy DataFrame and replace imputed values
df[numerical_columns] = imputed_values


In [97]:
populations = pd.read_excel('SUB-IP-EST2022-POP.xlsx')
# Rename the third column to "Population"
populations.rename(columns={populations.columns[4]: 'Population'}, inplace=True)
populations.rename(columns={populations.columns[0]: 'City'}, inplace=True)
populations['City'] = populations['City'].str.replace(r'\s(city|town)', '', regex=True)
# Display the DataFrame to verify the changes
print(populations.head())


                                                City  \
0  Annual Estimates of the Resident Population fo...   
1                                    Geographic Area   
2                                                NaN   
3                                 Abbeville, Alabama   
4                                Adamsville, Alabama   

                      Unnamed: 1                          Unnamed: 2  \
0                            NaN                                 NaN   
1  April 1, 2020\nEstimates Base  Population Estimate (as of July 1)   
2                            NaN                                2020   
3                           2355                                2356   
4                           4372                                4360   

   Unnamed: 3  Population  
0         NaN         NaN  
1         NaN         NaN  
2      2021.0      2022.0  
3      2361.0      2366.0  
4      4292.0      4224.0  


In [99]:
from geopy.geocoders import Photon

# Initialize Nominatim geocoder
geolocator = Photon(user_agent="geoapiExercises")

# Create empty lists to store latitude, longitude, and population data
latitudes, longitudes, hq_latitudes, hq_longitudes = [], [], [], []

# Loop through each city in the 'Location' column
for city_state in df['Location']:
    # Get location details
    location = geolocator.geocode(city_state)

    if location:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        latitudes.append(None)
        longitudes.append(None)
        
# Loop through each city in the 'Headquarters' column
for city_state in df['Headquarters']:
    # Get location details
    location = geolocator.geocode(city_state)

    if location:
        hq_latitudes.append(location.latitude)
        hq_longitudes.append(location.longitude)
    else:
        hq_latitudes.append(None)
        hq_longitudes.append(None)

# Add new columns to the DataFrame
df['Latitude'] = latitudes
df['Longitude'] = longitudes
df['HQ_Latitude'] = hq_latitudes
df['HQ_Longitude'] = hq_longitudes

print(df)

     Unnamed: 0                                          Job Title  \
0           0.0                                     Data Scientist   
1           1.0                          Healthcare Data Scientist   
2           2.0                                     Data Scientist   
3           3.0                                     Data Scientist   
4           4.0                                     Data Scientist   
..          ...                                                ...   
737       737.0           Sr Scientist, Immuno-Oncology - Oncology   
738       738.0                               Senior Data Engineer   
739       739.0  Project Scientist - Auton Lab, Robotics Institute   
740       740.0                               Data Science Manager   
741       741.0          Research Scientist – Security and Privacy   

                 Salary Estimate  \
0     $53K-$91K (Glassdoor est.)   
1    $63K-$112K (Glassdoor est.)   
2     $80K-$90K (Glassdoor est.)   
3     $56K-$97K

In [100]:

state_mapping = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire',
    'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina',
    'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
    'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee',
    'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}



    
def map_states(location):
    match = re.match(r'^(.*), ([A-Z]{2})$', location)
    if match:
        city, state_abbr = match.groups()
        mapped_state = state_mapping.get(state_abbr, '')
        return f"{city}, {mapped_state}"
    return location

# Apply mapping function to Location and Headquarters columns
df['Location'] = df['Location'].apply(map_states)
df['Headquarters'] = df['Headquarters'].apply(map_states)

merged_data = pd.merge(df, populations, left_on='Location', right_on='City', how='left')
merged_data = pd.merge(merged_data, populations, left_on='Headquarters', right_on='City', how='left')
print(merged_data.head())

   Unnamed: 0                  Job Title              Salary Estimate  \
0         0.0             Data Scientist   $53K-$91K (Glassdoor est.)   
1         1.0  Healthcare Data Scientist  $63K-$112K (Glassdoor est.)   
2         2.0             Data Scientist   $80K-$90K (Glassdoor est.)   
3         3.0             Data Scientist   $56K-$97K (Glassdoor est.)   
4         4.0             Data Scientist  $86K-$143K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Data Scientist\nLocation: Albuquerque, NM\nEdu...     3.8   
1  What You Will Do:\n\nI. General Summary\n\nThe...     3.4   
2  KnowBe4, Inc. is a high growth information sec...     4.8   
3  *Organization and Job ID**\nJob ID: 310709\n\n...     3.8   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

                                 Company Name                 Location  \
0                      Tecolote Research\n3.8  Albuquerque, New Mexico   
1  University of Maryland Me

In [101]:


df = merged_data
df['Log_Population'] = np.log(df['Population_x'])
df['Log_HQ_Population'] = np.log(df['Population_y'])

print(df)

     Unnamed: 0                                          Job Title  \
0           0.0                                     Data Scientist   
1           1.0                          Healthcare Data Scientist   
2           2.0                                     Data Scientist   
3           3.0                                     Data Scientist   
4           4.0                                     Data Scientist   
..          ...                                                ...   
737       737.0           Sr Scientist, Immuno-Oncology - Oncology   
738       738.0                               Senior Data Engineer   
739       739.0  Project Scientist - Auton Lab, Robotics Institute   
740       740.0                               Data Science Manager   
741       741.0          Research Scientist – Security and Privacy   

                 Salary Estimate  \
0     $53K-$91K (Glassdoor est.)   
1    $63K-$112K (Glassdoor est.)   
2     $80K-$90K (Glassdoor est.)   
3     $56K-$97K

In [102]:
# Assuming 'df' is your DataFrame
columns_to_remove = ['Unnamed: 3_x','Unnamed: 2_x','Unnamed: 1_x', 'Population_x', 'Unnamed: 3_y','Unnamed: 2_y','Unnamed: 1_y', 'Population_y']

# Remove specified columns
df = df.drop(columns=columns_to_remove)

In [103]:
# Filling NaN values with -9999 for attitude, longitude, hq_lattitude, hq_longitude, Log_Population, and Log_HQ_Population
columns_to_fill_na = ['Latitude', 'Longitude', 'HQ_Latitude', 'HQ_Longitude', 'Log_Population', 'Log_HQ_Population']
df[columns_to_fill_na] = df[columns_to_fill_na].fillna(-9999)

# Identifying numerical columns for imputation
numerical_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()

# Identifying rows for imputation
rows_for_imputation = df[columns_to_fill_na].isna().any(axis=1) | (df[columns_to_fill_na] == -9999).any(axis=1)

# Initializing KNNImputer
imputer = KNNImputer(missing_values=-9999, n_neighbors=10)

# Performing KNN imputation on NaN values for selected numerical columns
imputed_values = imputer.fit_transform(df[numerical_columns][rows_for_imputation])

# Copy DataFrame and replace imputed values
df.loc[rows_for_imputation, numerical_columns] = imputed_values

print(df)

     Unnamed: 0                                          Job Title  \
0           0.0                                     Data Scientist   
1           1.0                          Healthcare Data Scientist   
2           2.0                                     Data Scientist   
3           3.0                                     Data Scientist   
4           4.0                                     Data Scientist   
..          ...                                                ...   
737       737.0           Sr Scientist, Immuno-Oncology - Oncology   
738       738.0                               Senior Data Engineer   
739       739.0  Project Scientist - Auton Lab, Robotics Institute   
740       740.0                               Data Science Manager   
741       741.0          Research Scientist – Security and Privacy   

                 Salary Estimate  \
0     $53K-$91K (Glassdoor est.)   
1    $63K-$112K (Glassdoor est.)   
2     $80K-$90K (Glassdoor est.)   
3     $56K-$97K

In [104]:
# Assuming 'df' is your DataFrame
columns_to_keep = [
    'Rating', 'Size', 'hourly', 'employer_provided', 'avg_salary', 'same_state', 'age',
    'python_yn', 'R_yn', 'spark', 'aws', 'excel', 'desc_len', 'num_comp', 'is_senior',
    'Revenue_Category', 'Latitude', 'Longitude', 'HQ_Latitude', 'HQ_Longitude',
    'Log_Population', 'Log_HQ_Population', 'Sector', 'Type of ownership', 'job_simp'
]

# Filter columns to keep
df = df.filter(columns_to_keep)
print(df)

     Rating      Size  hourly  employer_provided  avg_salary  same_state  \
0       3.8  6.907755     0.0                0.0    0.243243         0.0   
1       3.4  9.210340     0.0                0.0    0.307692         0.0   
2       4.8  6.907755     0.0                0.0    0.297297         1.0   
3       3.8  8.517193     0.0                0.0    0.261954         1.0   
4       2.9  5.298317     0.0                0.0    0.419958         1.0   
..      ...       ...     ...                ...         ...         ...   
737     3.9  9.210340     0.0                0.0    0.295218         0.0   
738     4.4  8.517193     0.0                0.0    0.370062         0.0   
739     2.6  6.907755     0.0                0.0    0.249480         1.0   
740     3.2  3.912023     0.0                0.0    0.474012         0.0   
741     3.6  6.907755     0.0                0.0    0.332640         0.0   

          age  python_yn  R_yn  spark  ...  Revenue_Category   Latitude  \
0    0.17328

In [105]:
# Assuming 'df' is your dataframe containing the columns 'job_simp', 'Sector', and 'Type of ownership'

# Get dummy variables for job_simp column
job_simp_dummies = pd.get_dummies(df['job_simp'], prefix='job_title')

# Get dummy variables for Sector column
sector_dummies = pd.get_dummies(df['Sector'], prefix='Sector')

# Get dummy variables for Type of ownership column
ownership_dummies = pd.get_dummies(df['Type of ownership'], prefix='Ownership')

# Concatenate the dummy variables with the original dataframe
df = pd.concat([df, job_simp_dummies, sector_dummies, ownership_dummies], axis=1)

# Drop the original categorical columns if needed
df.drop(['job_simp', 'Sector', 'Type of ownership'], axis=1, inplace=True)

# Display the updated dataframe with dummy variables
print(df)

     Rating      Size  hourly  employer_provided  avg_salary  same_state  \
0       3.8  6.907755     0.0                0.0    0.243243         0.0   
1       3.4  9.210340     0.0                0.0    0.307692         0.0   
2       4.8  6.907755     0.0                0.0    0.297297         1.0   
3       3.8  8.517193     0.0                0.0    0.261954         1.0   
4       2.9  5.298317     0.0                0.0    0.419958         1.0   
..      ...       ...     ...                ...         ...         ...   
737     3.9  9.210340     0.0                0.0    0.295218         0.0   
738     4.4  8.517193     0.0                0.0    0.370062         0.0   
739     2.6  6.907755     0.0                0.0    0.249480         1.0   
740     3.2  3.912023     0.0                0.0    0.474012         0.0   
741     3.6  6.907755     0.0                0.0    0.332640         0.0   

          age  python_yn  R_yn  spark  ...  Ownership_College / University  \
0    0.17

In [106]:
# Replace True/False values in dummy variable columns with 1/0
dummy_columns = ['job_title', 'Sector', 'Ownership']  # Replace with your actual column names
for col in df.columns:
    if any(col.startswith(dc) for dc in dummy_columns):
        df[col] = df[col].astype(int)

# Display the updated dataframe with dummy variables
print(df)

     Rating      Size  hourly  employer_provided  avg_salary  same_state  \
0       3.8  6.907755     0.0                0.0    0.243243         0.0   
1       3.4  9.210340     0.0                0.0    0.307692         0.0   
2       4.8  6.907755     0.0                0.0    0.297297         1.0   
3       3.8  8.517193     0.0                0.0    0.261954         1.0   
4       2.9  5.298317     0.0                0.0    0.419958         1.0   
..      ...       ...     ...                ...         ...         ...   
737     3.9  9.210340     0.0                0.0    0.295218         0.0   
738     4.4  8.517193     0.0                0.0    0.370062         0.0   
739     2.6  6.907755     0.0                0.0    0.249480         1.0   
740     3.2  3.912023     0.0                0.0    0.474012         0.0   
741     3.6  6.907755     0.0                0.0    0.332640         0.0   

          age  python_yn  R_yn  spark  ...  Ownership_College / University  \
0    0.17

In [107]:
from sklearn.preprocessing import MinMaxScaler

# List of numerical columns to be scaled
numerical_columns = ['Rating', 'Size', 'hourly', 'employer_provided', 'avg_salary',
                     'same_state', 'age', 'python_yn', 'R_yn', 'spark', 'aws',
                     'excel', 'desc_len', 'num_comp', 'is_senior', 'Revenue_Category',
                     'Latitude', 'Longitude', 'HQ_Latitude', 'HQ_Longitude',
                     'Log_Population', 'Log_HQ_Population']

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale numerical columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

print(df)

       Rating      Size  hourly  employer_provided  avg_salary  same_state  \
0    0.800000  0.750000     0.0                0.0    0.243243         0.0   
1    0.733333  1.000000     0.0                0.0    0.307692         0.0   
2    0.966667  0.750000     0.0                0.0    0.297297         1.0   
3    0.800000  0.924743     0.0                0.0    0.261954         1.0   
4    0.650000  0.575257     0.0                0.0    0.419958         1.0   
..        ...       ...     ...                ...         ...         ...   
737  0.816667  1.000000     0.0                0.0    0.295218         0.0   
738  0.900000  0.924743     0.0                0.0    0.370062         0.0   
739  0.600000  0.750000     0.0                0.0    0.249480         1.0   
740  0.700000  0.424743     0.0                0.0    0.474012         0.0   
741  0.766667  0.750000     0.0                0.0    0.332640         0.0   

          age  python_yn  R_yn  spark  ...  Ownership_College /

In [108]:
df.drop('avg_salary', axis=1, inplace=True)

In [109]:
# Write the scaled DataFrame to a CSV file
df.to_csv('manual_features.csv', index=False)